# Displacement field NWES
Following the tutorial of https://docs.oceanparcels.org/en/latest/examples/documentation_unstuck_Agrid.html   
we create a displacement field for the CMEMS data of NWES https://data.marine.copernicus.eu/product/NWSHELF_ANALYSISFORECAST_PHY_004_013/description  
we use the NOwsystems (spain) NWES data (from september 2023 - nov 2025 (higher resolution))
But also do make it for the met-office data 

In [ ]:
#update reading in packages when rerunning this cell
%load_ext autoreload
%autoreload 2

from copy import copy

import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import numpy.ma as ma
import xarray as xr
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
from netCDF4 import Dataset
from scipy import interpolate

#import functions
import sys

sys.path.append("/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/src")
import displacement_field_functions as df 



In [ ]:
# set directories and files
base_directory_in = '/storage/shared/oceanparcels/input_data/CopernicusMarineService/'
filename_in_NOWsystems = 'OLD_NWSHELF_ANALYSIS_FORECAST_PHY_004_013/CMEMS_v6r1_NWS_PHY_NRT_NL_01hav3D_20230901_20230901_R20230902_HC01.nc'
filename_in_Metoffice = 'NWSHELF_ANALYSISFORECAST_PHY_004_013/cmems_mod_nws_phy-cur_anfc_1.5km-3D_PT1H-i_202511/metoffice_foam1_amm15_NWS_CUR_b20230903_hi20230901.nc' 
# there is a old (till nov 2023) and new (from sep 2023) version
versions = np.array(['Met','NOW'])
models = {}
models['Met'] = {}
models['NOW'] = {}
data_NOW = xr.open_dataset(base_directory_in+filename_in_NOWsystems)
data_met = xr.open_dataset(base_directory_in+filename_in_Metoffice)
models['Met']['lon'] = data_met['lon'].values
models['Met']['lat'] = data_met['lat'].values
models['NOW']['lon'] = data_NOW['longitude'].values
models['NOW']['lat'] = data_NOW['latitude'].values

print(data_NOW.attrs['source'])
print(data_met.attrs['source'])


In [ ]:
# create landmask, distance to shore and displacement field 

models['NOW']['landmask'] = df.make_landmask(base_directory_in+filename_in_NOWsystems)
models['Met']['landmask'] = df.make_landmask(base_directory_in+filename_in_Metoffice)

for vers in versions:
    models[vers]['d_2_s'] = df.distance_to_shore(models[vers]['landmask']) 
    v_x, v_y = df.create_displacement_field(models[vers]['landmask'])

    models[vers]['v_x'] = v_x
    models[vers]['v_y'] = v_y



In [ ]:
# save fields to netcdf file
output_directory = '/storage/shared/oceanparcels/output_data/data_Meike/NWES/'
for vers in versions:
    dset = xr.Dataset(
    data_vars=dict(dispU=(['lat','lon'],models[vers]['v_x']),
                   dispV=(['lat','lon'],models[vers]['v_y']),
                   landmask=(['lat','lon'],models[vers]['landmask']),
                   distance2shore=(['lat','lon'],models[vers]['d_2_s'])),
                    coords=dict(
                            lon=('lon',models[vers]['lon']),
                            lat=('lat',models[vers]['lat'])
                    ),
                    attrs=dict(
                        title='anti beaching fields NWES ' + vers,
                        contact = 'm.f.bos@uu.nl')
    )
    dset.to_netcdf(output_directory+'anti_beaching_NWES_'+vers+'.nc')

## Plotting fields
below we provide the stript to plot the landmasks and the fields 

In [ ]:
for vers in versions:
    models[vers]['dlon'] = models[vers]['lon'][11] -  models[vers]['lon'][10]
    models[vers]['dlat'] = models[vers]['lat'][11] -  models[vers]['lat'][10]
    models[vers]['lonmin'] = 200 
    models[vers]['lonmax'] = 220
    models[vers]['latmin'] = 200
    models[vers]['latmax'] = 220



In [ ]:
# for plotting
for vers in versions:
    lon_vals, lat_vals = np.meshgrid(models[vers]['lon'], models[vers]['lat'])
    models[vers]['lon_vals']=lon_vals
    models[vers]['lat_vals']=lat_vals
    models[vers]['lons_plot'] = lon_vals[models[vers]['latmin']:models[vers]['latmax'], models[vers]['lonmin']:models[vers]['lonmax']]
    models[vers]['lats_plot'] = lat_vals[models[vers]['latmin']:models[vers]['latmax'], models[vers]['lonmin']:models[vers]['lonmax']]



    x = models[vers]['lon'][:-1] + np.diff(models[vers]['lon']) / 2
    y = models[vers]['lat'][:-1] + np.diff(models[vers]['lat']) / 2
    lon_centers, lat_centers = np.meshgrid(x, y)
    models[vers]['lon_centers'] = lon_centers
    models[vers]['lat_centers'] = lat_centers

color_land = copy(plt.get_cmap("Reds"))(0)
color_ocean = copy(plt.get_cmap("Reds"))(128)



In [ ]:
# Interpolate the landmask to the cell centers
# only cells with 4 neighboring land points will be land
fl = interpolate.RectBivariateSpline(
    models['NOW']["lat"], models['NOW']["lon"], models['NOW']['landmask'], kx=1, ky=1
)

models['NOW']['l_centers'] = fl(models['NOW']['lat_centers'][:, 0], models['NOW']['lon_centers'][0, :])

# land when interpolated value == 1
models['NOW']['lmask']  = np.ma.masked_values(models['NOW']['l_centers'] , 1)

In [ ]:
fig = plt.figure(figsize=(12, 5))
fig.suptitle("Figure 1. Landmask", fontsize=18, y=1.01)
gs = gridspec.GridSpec(ncols=2, nrows=1, figure=fig)

ax0 = fig.add_subplot(gs[0, 0])
ax0.set_title("A) lazy use of pcolormesh", fontsize=11)
ax0.set_ylabel("Latitude [degrees]")
ax0.set_xlabel("Longitude [degrees]")

land0 = ax0.pcolormesh(
    models['NOW']['lons_plot'] ,
    models['NOW']['lats_plot'] ,
    models['NOW']['landmask'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    cmap="Reds_r",
    shading="auto",
)
ax0.scatter(
    models['NOW']['lons_plot'] ,
    models['NOW']['lats_plot'],
    c=models['NOW']['landmask'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    s=20,
    cmap="Reds_r",
    vmin=-0.05,
    vmax=0.05,
    edgecolors="k",
)

custom_lines = [
    Line2D(
        [0], [0], c=color_ocean, marker="o", markersize=10, markeredgecolor="k", lw=0
    ),
    Line2D(
        [0], [0], c=color_land, marker="o", markersize=10, markeredgecolor="k", lw=0
    ),
]
ax0.legend(
    custom_lines,
    ["ocean point", "land point"],
    bbox_to_anchor=(0.01, 0.93),
    loc="center left",
    borderaxespad=0.0,
    framealpha=1,
)

ax1 = fig.add_subplot(gs[0, 1])
ax1.set_title("B) correct A grid representation in Parcels", fontsize=11)
ax1.set_ylabel("Latitude [degrees]")
ax1.set_xlabel("Longitude [degrees]")

land1 = ax1.pcolormesh(
    models['NOW']['lon_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lat_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lmask'].mask[ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    cmap="Reds_r",
)
ax1.scatter(
    models['NOW']['lons_plot'],
    models['NOW']['lats_plot'],
    c=models['NOW']['landmask'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    s=20,
    cmap="Reds_r",
    vmin=-0.05,
    vmax=0.05,
    edgecolors="k",
)

ax1.legend(
    custom_lines,
    ["ocean point", "land point"],
    bbox_to_anchor=(0.01, 0.93),
    loc="center left",
    borderaxespad=0.0,
    framealpha=1,
);

In [ ]:
models['NOW']['coastal'] = df.get_coastal_nodes_diagonal(models['NOW']['landmask'])
models['NOW']['shore'] = df.get_shore_nodes_diagonal(models['NOW']['landmask'])

In [ ]:
fig = plt.figure(figsize=(10, 4), constrained_layout=True)
fig.suptitle("Figure 2. Coast and Shore", fontsize=18, y=1.04)
gs = gridspec.GridSpec(ncols=2, nrows=1, figure=fig)


ax0 = fig.add_subplot(gs[0, 0])
land0 = ax0.pcolormesh(
    models['NOW']['lon_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lat_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lmask'].mask[ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    cmap="Reds_r",
)
coa = ax0.scatter(
    models['NOW']['lons_plot'], models['NOW']['lats_plot'], c=models['NOW']['coastal'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']], cmap="Reds_r", s=50
)
ax0.scatter(
     models['NOW']['lons_plot'],
     models['NOW']['lats_plot'],
    c=models['NOW']['landmask'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    s=20,
    cmap="Reds_r",
    vmin=-0.05,
    vmax=0.05,
)

ax0.set_title("Coast")
ax0.set_ylabel("Latitude [degrees]")
ax0.set_xlabel("Longitude [degrees]")

custom_lines = [
    Line2D([0], [0], c=color_ocean, marker="o", markersize=5, lw=0),
    Line2D(
        [0],
        [0],
        c=color_ocean,
        marker="o",
        markersize=7,
        markeredgecolor="w",
        markeredgewidth=2,
        lw=0,
    ),
    Line2D(
        [0],
        [0],
        c=color_land,
        marker="o",
        markersize=7,
        markeredgecolor="firebrick",
        lw=0,
    ),
]
ax0.legend(
    custom_lines,
    ["ocean node", "coast node", "land node"],
    bbox_to_anchor=(0.01, 0.9),
    loc="center left",
    borderaxespad=0.0,
    framealpha=1,
    facecolor="silver",
)


ax1 = fig.add_subplot(gs[0, 1])
land1 = ax1.pcolormesh(
    models['NOW']['lon_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lat_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lmask'].mask[ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    cmap="Reds_r",
)
sho = ax1.scatter(
    models['NOW']['lons_plot'], models['NOW']['lats_plot'], c=models['NOW']['shore'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']], cmap="Reds_r", s=50
)
ax1.scatter(
    models['NOW']['lons_plot'],
    models['NOW']['lats_plot'],
    c=models['NOW']['landmask'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    s=20,
    cmap="Reds_r",
    vmin=-0.05,
    vmax=0.05,
)

ax1.set_title("Shore")
ax1.set_ylabel("Latitude [degrees]")
ax1.set_xlabel("Longitude [degrees]")

custom_lines = [
    Line2D([0], [0], c=color_ocean, marker="o", markersize=5, lw=0),
    Line2D(
        [0],
        [0],
        c=color_land,
        marker="o",
        markersize=7,
        markeredgecolor="w",
        markeredgewidth=2,
        lw=0,
    ),
    Line2D(
        [0],
        [0],
        c=color_land,
        marker="o",
        markersize=7,
        markeredgecolor="firebrick",
        lw=0,
    ),
]
ax1.legend(
    custom_lines,
    ["ocean node", "shore node", "land node"],
    bbox_to_anchor=(0.01, 0.9),
    loc="center left",
    borderaxespad=0.0,
    framealpha=1,
    facecolor="silver",
);



In [ ]:
v_x, v_y = df.create_displacement_field(models['NOW']['landmask'])
models['NOW']['v_x'] = v_x
models['NOW']['v_y'] = v_y

In [ ]:
fig = plt.figure(figsize=(7, 6), constrained_layout=True)
fig.suptitle("Figure 3. Displacement field", fontsize=18, y=1.04)
gs = gridspec.GridSpec(ncols=1, nrows=1, figure=fig)

ax0 = fig.add_subplot(gs[0, 0])
land = ax0.pcolormesh(
    models['NOW']['lon_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lat_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lmask'].mask[ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    cmap="Reds_r",
)
ax0.scatter(
    models['NOW']['lons_plot'],
    models['NOW']['lats_plot'],
    c=models['NOW']['landmask'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    s=30,
    cmap="Reds_r",
    vmin=-0.05,
    vmax=0.05,
    edgecolors="k",
)
quiv = ax0.quiver(
    models['NOW']['lons_plot'],
    models['NOW']['lats_plot'],
    models['NOW']['v_x'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    models['NOW']['v_y'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    color="orange",
    angles="xy",
    scale_units="xy",
    scale=19,
    width=0.005,
)

ax0.set_ylabel("Latitude [degrees]")
ax0.set_xlabel("Longitude [degrees]")

custom_lines = [
    Line2D(
        [0], [0], c=color_ocean, marker="o", markersize=10, markeredgecolor="k", lw=0
    ),
    Line2D(
        [0], [0], c=color_land, marker="o", markersize=10, markeredgecolor="k", lw=0
    ),
]
ax0.legend(
    custom_lines,
    ["ocean point", "land point"],
    bbox_to_anchor=(0.01, 0.93),
    loc="center left",
    borderaxespad=0.0,
    framealpha=1,
);

In [ ]:
models['NOW']['d_2_s'] = df.distance_to_shore(models['NOW']['landmask'])

In [ ]:
fig = plt.figure(figsize=(6, 5), constrained_layout=True)

ax0 = fig.add_subplot()
ax0.set_title("Figure 4. Distance to shore", fontsize=18)
ax0.set_ylabel("Latitude [degrees]")
ax0.set_xlabel("Longitude [degrees]")

land = ax0.pcolormesh(
    models['NOW']['lon_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lat_vals'][ models['NOW']['latmin']: models['NOW']['latmax']+1,  models['NOW']['lonmin']: models['NOW']['lonmax']+1],
    models['NOW']['lmask'].mask[ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']],
    cmap="Reds_r",
)
d2s = ax0.scatter(models['NOW']['lons_plot'], models['NOW']['lats_plot'], c=models['NOW']['d_2_s'][ models['NOW']['latmin']: models['NOW']['latmax'],  models['NOW']['lonmin']: models['NOW']['lonmax']])

plt.colorbar(d2s, ax=ax0, label="Distance [gridcells]");



# Make displacement field for daily field

In [ ]:
file_daily_in = '/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/input_data/cmems_mod_nws_phy_anfc_0.027deg-daily_field_2023-09-01_2024-03-06.nc'

# there is a old (till nov 2023) and new (from sep 2023) version
versions = np.array(['Met','NOW'])

models['daily'] = {}
data_daily = xr.open_dataset(file_daily_in)
data_daily = data_daily.isel(time = 0)

models['daily']['lon'] = data_daily['longitude'].values
models['daily']['lat'] = data_daily['latitude'].values
print(data_daily.attrs['source'])


In [ ]:
models['daily']['landmask'] = df.make_landmask(file_daily_in)
models['daily']['d_2_s'] = df.distance_to_shore(models['daily']['landmask']) 
v_x, v_y = df.create_displacement_field(models['daily']['landmask'])
models['daily']['v_x'] = v_x
models['daily']['v_y'] = v_y

In [ ]:
# save fields to netcdf file
output_directory = '/nethome/4291387/Maxey_Riley_advection/Maxey_Riley_advection/input_data/'
vers = 'daily'
dset = xr.Dataset(
data_vars=dict(dispU=(['lat','lon'],models[vers]['v_x']),
                dispV=(['lat','lon'],models[vers]['v_y']),
                landmask=(['lat','lon'],models[vers]['landmask']),
                distance2shore=(['lat','lon'],models[vers]['d_2_s'])),
                coords=dict(
                        lon=('lon',models[vers]['lon']),
                        lat=('lat',models[vers]['lat'])
                ),
                attrs=dict(
                    title='anti beaching fields NWES ' + vers,
                    contact = 'm.f.bos@uu.nl')
    )
dset.to_netcdf(output_directory+'anti_beaching_NWES_NOW_'+vers+'.nc')